In [24]:
%matplotlib inline

from __future__ import print_function

import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm


# Fit guassian to 1332 peak
from scipy.optimize import curve_fit

def gauss_function(x, a, x0, sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))


In [25]:
# Find Gaussians
intervals = []

df = pd.read_csv('data/time.csv')

gaussians = []
prev = False
gaussian = []
for index,row in df.iterrows():
    if row['Counts'] > 0:
        gaussian.append((row['Channel'], row['Counts']))
        prev = True
        continue
    if row['Counts'] == 0 and prev == True:
        if len(gaussian) > 1:
            gaussians.append(gaussian)
        gaussian = []
    prev = False     

In [27]:
def fit_gaussian(data):
    print(hi)